In [1]:
# Importamos las librerias

import pandas as pd
import numpy as np

In [160]:
# Vamos a crear un diccionario con los diferentes nombres de los archivos a procesar

# Definimos los nombres
nombres = [
    "archivos_csv/Analisis de Datos Dengue y Zika - 2018.csv",
    "archivos_csv/Analisis de Datos Dengue y Zika - 2019.csv",
    "archivos_csv/Analisis de Datos Dengue y Zika - 2020.csv",
    "archivos_csv/Analisis de Datos Dengue y Zika - Julio 2021.csv",
    "archivos_csv/Analisis de Datos Dengue y Zika - Diciembre 2021.csv",
    "archivos_csv/Analisis de Datos Dengue y Zika - 2022.csv",
    "archivos_csv/Analisis de Datos Dengue y Zika - 2023.csv",
]

# Ahora las columnas que usaremos de cada archivo
columnas = [
    ["provincia_id", "provincia_nombre", "anio", "semanas_epidemiologicas", "evento_nombre", "cantidad_casos"],
    ["provincia_id", "provincia_nombre", "año", "semanas_epidemiologicas", "evento_nombre", "cantidad_casos"],
    ["provincia_id", "provincia_nombre", "año", "semanas_epidemiologicas", "evento_nombre", "cantidad_casos"],
    ["provincia_id", "provincia_nombre", "año", "semanas_epidemiologicas", "evento_nombre", "cantidad_casos"],
    ["provincia_nombre", "provincia_id", "año", "semanas_epidemiologicas", "evento_nombre", "cantidad_casos"],
    ["provincia_id", "provincia_nombre", "año", "semanas_epidemiologicas", "evento_nombre", "cantidad_casos"],
    ["ID_PROV_INDEC_RESIDENCIA", "PROVINCIA_RESIDENCIA", "ANIO_MIN", "SEPI_MIN", "EVENTO", "Total"],
            ]

# Conformamos el diccionario
diccionario_archivos = {}

for nombre, columnas in zip(nombres, columnas):
    diccionario_archivos[nombre] = {"nombre": nombre, "columnas": columnas}


def procesar_csv(archivo_csv, columnas):
    # Cargamos el dataset
    df = pd.read_csv(archivo_csv, usecols=columnas)

    # Renombramos las columnas
    columnas_renombradas = {
        columnas[0]: "id_provincia",
        columnas[1]: "nombre_provincia",
        columnas[2]: "anio",
        columnas[3]: "semana_epidemiologica",
        columnas[4]: "nombre_evento",
        columnas[5]: "total_casos"
        }
    
    df.rename(columns=columnas_renombradas, inplace=True)

    # Completamos aquellos registros que no tengan cargados datos en el "nombre_provincia" con "Sin Especificar"
    df["nombre_provincia"] = df["nombre_provincia"].fillna("Sin Especificar")

    # Lo mismo para los que tienen *sin dato*
    df["nombre_provincia"] = df["nombre_provincia"].str.replace("*sin dato*", "Sin Especificar")

    # Tambien para los que tienen (en blanco)
    df["nombre_provincia"] = df["nombre_provincia"].str.replace("(en blanco)", "Sin Especificar")

    # Corregimos errores de escritura en los datos
    df["nombre_provincia"] = df["nombre_provincia"].str.replace("SAnta Cruz", "Santa Cruz")
    df["nombre_provincia"] = df["nombre_provincia"].str.replace("SAnta Fe", "Santa Fe")


    # Reemplazamos las tildes para que el resultado sea optimo
    df["nombre_provincia"] = df["nombre_provincia"].map(lambda x: x.replace("á", "a"))
    df["nombre_provincia"] = df["nombre_provincia"].map(lambda x: x.replace("é", "e"))
    df["nombre_provincia"] = df["nombre_provincia"].map(lambda x: x.replace("í", "i"))
    df["nombre_provincia"] = df["nombre_provincia"].map(lambda x: x.replace("ó", "o"))

    # Filtrar por "nombre_evento" == "Dengue"
    df = df.loc[df["nombre_evento"] == "Dengue"]

    # Agrupar por "id_provincia", "provincia_nombre", "anio", "evento_nombre", "semanas_epidemiologicas"
    df_agrupado = df.groupby(["id_provincia", "nombre_provincia", "anio", "semana_epidemiologica"])

    # Calcular el total de casos por semana
    df = df_agrupado["total_casos"].agg("sum")

    # Convertir el Series a un DataFrame
    df_procesado = df.to_frame()

    print("✅ Archivo Procesado:", archivo_csv)
    print()

    return df_procesado

# Vamos a analizar todos los datasets secuencialmente, e iremos guardando los archivos parciales en una lista
df_temporal = []

for nombre_archivo in diccionario_archivos:
    columnas = diccionario_archivos[nombre_archivo]["columnas"]
    nombre = diccionario_archivos[nombre_archivo]["nombre"]
    df = procesar_csv(nombre, columnas)
    df_temporal.append(df)

df = pd.concat(df_temporal)

df["total_casos"].astype("int")

df.to_csv("archivos_csv/casos_dengue_2018_2023.csv")

print("🚨 Los archivos han sido procesados exitosamente!")

✅ Archivo Procesado: archivos_csv/Analisis de Datos Dengue y Zika - 2018.csv

✅ Archivo Procesado: archivos_csv/Analisis de Datos Dengue y Zika - 2019.csv

✅ Archivo Procesado: archivos_csv/Analisis de Datos Dengue y Zika - 2020.csv

✅ Archivo Procesado: archivos_csv/Analisis de Datos Dengue y Zika - Julio 2021.csv

✅ Archivo Procesado: archivos_csv/Analisis de Datos Dengue y Zika - Diciembre 2021.csv

✅ Archivo Procesado: archivos_csv/Analisis de Datos Dengue y Zika - 2022.csv

✅ Archivo Procesado: archivos_csv/Analisis de Datos Dengue y Zika - 2023.csv

🚨 Los archivos han sido procesados exitosamente!
